In [1]:
import matplotlib
matplotlib.use("Agg")
import json
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
# import keras
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import Callback
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.regularizers import l1
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MultiLabelBinarizer
from cnn_model import ImageClassificationModel

from PIL import Image
import matplotlib.pyplot as plt
import os
import pickle as pkl
import dask.array as da
import numpy as np

Using TensorFlow backend.


In [2]:
target_size = (128,128)
train_data_dir = 'train'
validation_data_dir = 'validation'
nb_train_samples = len(os.listdir(train_data_dir))
nb_validation_samples = len(os.listdir(validation_data_dir))
epochs = 50
batch_size = 60

In [3]:
with open('annotations/instances_train2014.json','r') as file:
    instances = json.load(file)

labels = np.array([category['name'] for category in instances['categories']])
del instances

In [5]:
train_x = ((np.load('train_x_128.npy', allow_pickle=True)).astype(int)).astype('uint8')
train_x = da.from_array(train_x, chunks = (2481,128,128,3))
train_y = np.load('train_y.npy', allow_pickle=True)
train_y = np.array(train_y)
mlb = MultiLabelBinarizer(labels)
train_y = mlb.fit_transform(train_y)

In [7]:
validation_x = ((np.load('validation_x_128.npy', allow_pickle=True)).astype(int)).astype('uint8')
validation_x = da.from_array(validation_x, chunks = (827,128,128,3))
validation_y = np.load('validation_y.npy', allow_pickle=True)
validation_y = np.array(validation_y)
mlb = MultiLabelBinarizer(labels)
validation_y = mlb.fit_transform(validation_y)

In [8]:
train_datagen = ImageDataGenerator(
        rotation_range=25, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        rescale=1./255,
        shear_range=0.2,
        fill_mode='nearest',
        zoom_range=0.2,
        horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)

model = ImageClassificationModel.build(target_size[0], target_size[1], len(labels), 'softmax')
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001, beta_1=0.9, beta_2=0.95), metrics=['accuracy'])
if "best_model.h5" in os.listdir():
    model = load_model("best_model.h5")

In [13]:
stop_early = EarlyStopping(monitor='val_loss', patience=10)
reduceLR = ReduceLROnPlateau(monitor='val_loss', paitence = 10, factor=0.2, min_lr = 0.0001)
ModelCheck = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
callbacks = [stop_early, reduceLR, ModelCheck]
model.fit_generator(
    train_datagen.flow(train_x, train_y, batch_size= batch_size),
    steps_per_epoch = nb_train_samples//batch_size,
    epochs = epochs,
    validation_data=validation_datagen.flow(validation_x, validation_y, batch_size=batch_size),
    validation_steps = nb_validation_samples//batch_size,
    use_multiprocessing = True,
    # initial_epoch = 0,
    callbacks = callbacks)

Epoch 1/50
826/827 [============================>.] - ETA: 0s - loss: 8.7584 - accuracy: 0.5247
Epoch 00001: val_accuracy improved from -inf to 0.53224, saving model to best_model.h5
827/827 [==============================] - 969s 1s/step - loss: 8.7596 - accuracy: 0.5247 - val_loss: 9.6822 - val_accuracy: 0.5322
Epoch 2/50
826/827 [============================>.] - ETA: 0s - loss: 8.7500 - accuracy: 0.5260
Epoch 00002: val_accuracy did not improve from 0.53224
827/827 [==============================] - 188s 228ms/step - loss: 8.7509 - accuracy: 0.5260 - val_loss: 9.7174 - val_accuracy: 0.5227
Epoch 3/50
826/827 [============================>.] - ETA: 0s - loss: 8.7478 - accuracy: 0.5246
Epoch 00003: val_accuracy did not improve from 0.53224
827/827 [==============================] - 196s 237ms/step - loss: 8.7480 - accuracy: 0.5247 - val_loss: 9.4422 - val_accuracy: 0.5318
Epoch 4/50
826/827 [============================>.] - ETA: 0s - loss: 8.7619 - accuracy: 0.5246 ETA: 0s - loss: 8

In [10]:
model.save("best_model.h5")
f = open('mlb.pickle', 'wb')
f.write(pkl.dumps(mlb))
f.close()

In [12]:
img = Image.open('test/COCO_train2014_000000349007.jpg') # image extension *.png,*.jpg
img = img.resize((128,128), Image.ANTIALIAS)
img.show()
arr = np.array(img)
arr=np.expand_dims(arr,0)
arr=arr/255
proba = model.predict(arr)[0]
proba= (proba*1000).astype(int)
for x in range(len(proba)):
    if proba[x] >100:
        proba[x]=1
    else:
        proba[x]=0

for i in range(len(proba)):
    if(proba[i]==1):
        print(labels[i])

person
pizza
dining table
